In [2]:
# python version > 3.10 or 3.11 
!pip3 install --upgrade google-cloud-aiplatform
!pip3 install --upgrade google-cloud-bigquery
!pip3 install --upgrade google-cloud-storage
!pip3 install --upgrade google-auth
# if already exist please ignore the installations

  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/95/a8/bc583352dd5020be9651582fe3ebfd46a1e3e2130cb09d638cecf1cd9842/google_cloud_aiplatform-1.74.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-resource-manager<3.0.0dev,>=1.3.3 from https://files.pythonhosted.org/packages/64/c4/2275ca35419f9a2ae66846f389490b356856bf55a9ad9f95a88399a89294/google_cloud_resource_manager-1.14.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for shapely<3.0.0dev from https://files.pythonhosted.org/packages/b1/5a/6a67d929c467a1973b6bb9f0b00159cc343b02bf9a8d26db1abd2f87aa23/shapely-2.0.6-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for docstring-parser<1 from https://files.pythonhosted.org/packages/d5/7c/e9fcff7623954d86bdc17782036cbf715ecab1bec4847c008557affe1ca8/docstring_parser-0.16-py3-none-any.whl.metadata
  Obtaining dependency information for grpc-google-iam-v1<1.0.0d

In [7]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform


## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle


In [8]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "trial_bigq.json"

In [9]:
project_id = 'dtidsus'
dataset_id = 'capstone'
table_id = 'data_ecommerce_customer_churn'
region = 'us-central1'
bucket_name = 'modul4'
blob_name = 'fine/data_ecommerce_customer_churn.csv'
client = bigquery.Client(project='dtidsus')

In [11]:
try : 
    storage_client = storage.Client(project='dtidsus')
    bucket = storage_client.get_bucket(bucket_name)
    data_capstone = bucket.blob('fine/data_ecommerce_customer_churn.csv')
    data_capstone.upload_from_filename(r'C:\Users\finePC\Desktop\DTI - PURWADHIKA\MODUL 3 - ML\CAPSTONE\cleaned_ecommerce.csv')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [12]:
# Upload to Google Cloud Platform

try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob('fine/model/model.pkl')
    blob_model.upload_from_filename('model.pkl')

    print("Uploading model succeeded")
except Exception as e:
    print("An exception occurred while uploading the model:", e)

Uploading model succeeded


In [13]:
from google.cloud import aiplatform

In [14]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "dev_trial.json"

In [15]:
aiplatform.init(project='dtidsus', location='us-central1')

model = aiplatform.Model.upload(
    display_name='jaya_model_000',
    artifact_uri="gs://modul4/fine/model/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

Creating Model
Create Model backing LRO: projects/41965541199/locations/us-central1/models/6781529334889840640/operations/6411212066308227072
Model created. Resource name: projects/41965541199/locations/us-central1/models/6781529334889840640@1
To use this Model in another session:
model = aiplatform.Model('projects/41965541199/locations/us-central1/models/6781529334889840640@1')


In [16]:
endpoint = aiplatform.Endpoint.create(
    display_name="fine-endpoint-000",
    project='dtidsus',
    location='us-central1',
)

Creating Endpoint
Create Endpoint backing LRO: projects/41965541199/locations/us-central1/endpoints/1834698978176794624/operations/1083453707128930304
Endpoint created. Resource name: projects/41965541199/locations/us-central1/endpoints/1834698978176794624
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/41965541199/locations/us-central1/endpoints/1834698978176794624')


In [17]:
min_replica_count: int = 1
max_replica_count: int = 1

In [18]:
endpoint.deploy( 
    model=model,
    deployed_model_display_name='fine_model_000',
    machine_type='e2-standard-2',
    min_replica_count=min_replica_count,
    max_replica_count=max_replica_count,
    sync=True, 
)

Deploying Model projects/41965541199/locations/us-central1/models/6781529334889840640 to Endpoint : projects/41965541199/locations/us-central1/endpoints/1834698978176794624
Deploy Endpoint model backing LRO: projects/41965541199/locations/us-central1/endpoints/1834698978176794624/operations/5379887751640383488


FailedPrecondition: 400 Model server exited unexpectedly. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=41965541199&resource=aiplatform.googleapis.com%2FEndpoint&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FEndpoint%22%0Aresource.labels.endpoint_id%3D%221834698978176794624%22%0Aresource.labels.location%3D%22us-central1%22. 9: Model server exited unexpectedly. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=41965541199&resource=aiplatform.googleapis.com%2FEndpoint&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FEndpoint%22%0Aresource.labels.endpoint_id%3D%221834698978176794624%22%0Aresource.labels.location%3D%22us-central1%22.

In [19]:
## predict your data with online prediction here 
PROJECT_ID = 'dtidsus'
ENDPOINT_ID = "projects/41965541199/locations/us-central1/endpoints/2241324766407426048"
REGION = 'us-central1'



In [20]:
aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint = aiplatform.Endpoint(ENDPOINT_ID)
prediction = endpoint.predict(instances=[[50000, 1, 3, 1, 23, 131895, 132154, 129237]])

print("PREDICTION:", prediction)

PREDICTION: Prediction(predictions=[0.0], deployed_model_id='6724688981780332544', metadata=None, model_version_id='1', model_resource_name='projects/41965541199/locations/us-central1/models/4379281146152747008', explanations=None)
